In [1]:
from tensorflow.keras.applications import InceptionV3
from tensorflow import keras
from tensorflow.keras.layers import *
from prepare_data import *
from datetime import datetime
from tensorflow.keras.applications.inception_v3 import preprocess_input
import albumentations as A

In [2]:
dataset = Dataset.carabid
_, val_dataset = prep_data_single(dataset, 1)


extractor_model_path = "models/carabid/CARABID-EXTRACTOR/extractor/savefile.hdf5"
extractor_model = keras.models.load_model(extractor_model_path)

augs = {"no augs": None,
        "horizontal flip": A.Compose([A.HorizontalFlip(p=1.0)]),
        "vertical flip": A.Compose([A.VerticalFlip(p=1.0)]),
        "rotate": A.Compose([A.Rotate(p=1.0)]),
        "brightness/contrast": A.Compose([A.RandomBrightnessContrast(p=1.0)]),
        "gamma": A.Compose([A.RandomGamma(p=1.0)]),
        "grey": A.Compose([A.ToGray(p=1.0)]),
        "blur": A.Compose([A.Blur(p=1.0)])}

Found 63364 files belonging to 291 classes.
Using 53860 files for training.
Found 63364 files belonging to 291 classes.
Using 9504 files for validation.


In [3]:
features_model = keras.Sequential([extractor_model.layers[0].layers[-1]])
features_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [4]:
feature_outputs = {}
generators = {}
for name, aug in augs.items():
    generators[name] = prep_data_aug_single(dataset, val_dataset, val_dataset, 1, augment=aug)[1]
    feature_outputs[name] = np.zeros(1000)

for i in range(len(generators["no augs"])):
    img, _ = generators["no augs"].__getitem__(i)
    ground_truth = features_model.predict(img)[0]

    for name, generator in generators.items():
        if name == "no augs":
            continue

        img, _ = generator.__getitem__(i)
        features = features_model.predict(img)[0]
        feature_outputs[name] += np.abs(ground_truth - features)

print(feature_outputs)
length = len(generators["no augs"])
for name, result in feature_outputs.items():
    print(f"{name}: {sum(result / length)}")

{'no augs': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0.

In [5]:
feature_outputs2 = {}
generators = {}

augs = {"no augs": None,
        "horizontal flip - vertical flip": A.Compose([A.HorizontalFlip(p=1.0), A.VerticalFlip(p=1.0)]),
        "horizontal flip - rotate": A.Compose([A.HorizontalFlip(p=1.0), A.Rotate(p=1.0)]),
        "horizontal flip - brightness/contrast": A.Compose([A.HorizontalFlip(p=1.0), A.RandomBrightnessContrast(p=1.0)]),
        "horizontal flip - gamma": A.Compose([A.HorizontalFlip(p=1.0), A.RandomGamma(p=1.0)]),
        "horizontal flip - grey": A.Compose([A.HorizontalFlip(p=1.0), A.ToGray(p=1.0)]),
        "horizontal flip - blur": A.Compose([A.HorizontalFlip(p=1.0), A.Blur(p=1.0)])}
for name, aug in augs.items():
    generators[name] = prep_data_aug_single(dataset, val_dataset, val_dataset, 1, augment=aug)[1]
    feature_outputs2[name] = np.zeros(1000)

for i in range(len(generators["no augs"])):
    img, _ = generators["no augs"].__getitem__(i)
    ground_truth = features_model.predict(img)[0]

    for name, generator in generators.items():
        if name == "no augs":
            continue

        img, _ = generator.__getitem__(i)
        features = features_model.predict(img)[0]
        feature_outputs2[name] += np.abs(ground_truth - features)

print(feature_outputs2)
length = len(generators["no augs"])
for name, result in feature_outputs2.items():
    print(f"{name}: {sum(result / length)}")

{'no augs': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0.

In [3]:
aug_results = {}
augs = {"no augs": None,
        "horizontal flip": A.Compose([A.HorizontalFlip(p=1.0)]),
        "vertical flip": A.Compose([A.VerticalFlip(p=1.0)]),
        "rotate": A.Compose([A.Rotate(p=1.0)]),
        "brightness/contrast": A.Compose([A.RandomBrightnessContrast(p=1.0)]),
        "gamma": A.Compose([A.RandomGamma(p=1.0)]),
        "grey": A.Compose([A.ToGray(p=1.0)]),
        "blur": A.Compose([A.Blur(p=1.0)])}
for name, aug in augs.items():
    _, extractor_val = prep_data_aug_single(dataset, val_dataset, val_dataset, 1, augment=aug)
    aug_results[name] = extractor_model.evaluate(extractor_val)

print(aug_results)

9504/9504 [==============================] - 156s 16ms/step - loss: 5.4790 - accuracy: 0.3787
{'no augs': [0.41842904686927795, 0.8894149661064148], 'horizontal flip': [0.41799378395080566, 0.8902567625045776], 'vertical flip': [0.4215128421783447, 0.8912037014961243], 'rotate': [0.4481838345527649, 0.8833122849464417], 'brightness/contrast': [0.9452727437019348, 0.7766203880310059], 'gamma': [0.4777602255344391, 0.8717381954193115], 'grey': [7.036161422729492, 0.11416245996952057], 'blur': [5.479002952575684, 0.3786826729774475]}


In [8]:
two_img_2aug_hori_results = {}
augs = {"no augs": None,
        "horizontal flip - vertical flip": A.Compose([A.HorizontalFlip(p=1.0), A.VerticalFlip(p=1.0)]),
        "horizontal flip - rotate": A.Compose([A.HorizontalFlip(p=1.0), A.Rotate(p=1.0)]),
        "horizontal flip - brightness/contrast": A.Compose([A.HorizontalFlip(p=1.0), A.RandomBrightnessContrast(p=1.0)]),
        "horizontal flip - gamma": A.Compose([A.HorizontalFlip(p=1.0), A.RandomGamma(p=1.0)]),
        "horizontal flip - grey": A.Compose([A.HorizontalFlip(p=1.0), A.ToGray(p=1.0)]),
        "horizontal flip - blur": A.Compose([A.HorizontalFlip(p=1.0), A.Blur(p=1.0)])}
for name, aug in augs.items():
    _, extractor_val = prep_data_aug_single(dataset, val_dataset, val_dataset, 1, augment=aug)
    two_img_2aug_hori_results[name] = extractor_model.evaluate(extractor_val)

print(two_img_2aug_hori_results)

9504/9504 [==============================] - 169s 18ms/step - loss: 5.5526 - accuracy: 0.3692
{'no augs': [0.41844695806503296, 0.8894149661064148], 'horizontal flip - vertical flip': [0.4158937633037567, 0.8910984992980957], 'horizontal flip - rotate': [0.4471031129360199, 0.8834174871444702], 'horizontal flip - brightness/contrast': [0.9313725233078003, 0.7771464586257935], 'horizontal flip - gamma': [0.48271411657333374, 0.8746843338012695], 'horizontal flip - grey': [6.969332695007324, 0.11447811126708984], 'horizontal flip - blur': [5.552605628967285, 0.36921295523643494]}


In [9]:
two_img_2aug_gamma_results = {}
augs = {"no augs": None,
        "gamma - vertical flip": A.Compose([A.RandomGamma(p=1.0), A.VerticalFlip(p=1.0)]),
        "gamma - rotate": A.Compose([A.RandomGamma(p=1.0), A.Rotate(p=1.0)]),
        "gamma - brightness/contrast": A.Compose([A.RandomGamma(p=1.0), A.RandomBrightnessContrast(p=1.0)]),
        "gamma - horizontal flip": A.Compose([A.RandomGamma(p=1.0), A.HorizontalFlip(p=1.0)]),
        "gamma - grey": A.Compose([A.RandomGamma(p=1.0), A.ToGray(p=1.0)]),
        "gamma - blur": A.Compose([A.RandomGamma(p=1.0), A.Blur(p=1.0)])}
for name, aug in augs.items():
    _, extractor_val = prep_data_aug_single(dataset, val_dataset, val_dataset, 1, augment=aug)
    two_img_2aug_gamma_results[name] = extractor_model.evaluate(extractor_val)

print(two_img_2aug_gamma_results)

9504/9504 [==============================] - 149s 16ms/step - loss: 5.6449 - accuracy: 0.3580
{'no augs': [0.4184473156929016, 0.8894149661064148], 'gamma - vertical flip': [0.4882015585899353, 0.8728955984115601], 'gamma - rotate': [0.519961953163147, 0.8638467788696289], 'gamma - brightness/contrast': [1.0163623094558716, 0.7640992999076843], 'gamma - horizontal flip': [0.48745959997177124, 0.8727903962135315], 'gamma - grey': [7.25425910949707, 0.10805976390838623], 'gamma - blur': [5.644865036010742, 0.3579545319080353]}


In [10]:
two_img_2aug_bricon_results = {}
augs = {"no augs": None,
        "brightness/contrast - vertical flip": A.Compose([A.RandomBrightnessContrast(p=1.0), A.VerticalFlip(p=1.0)]),
        "brightness/contrast - rotate": A.Compose([A.RandomBrightnessContrast(p=1.0), A.Rotate(p=1.0)]),
        "brightness/contrast - horizontal flip": A.Compose([A.RandomBrightnessContrast(p=1.0), A.HorizontalFlip(p=1.0)]),
        "brightness/contrast - gamma": A.Compose([A.RandomBrightnessContrast(p=1.0), A.RandomGamma(p=1.0)]),
        "brightness/contrast - grey": A.Compose([A.RandomBrightnessContrast(p=1.0), A.ToGray(p=1.0)]),
        "brightness/contrast - blur": A.Compose([A.RandomBrightnessContrast(p=1.0), A.Blur(p=1.0)])}
for name, aug in augs.items():
    _, extractor_val = prep_data_aug_single(dataset, val_dataset, val_dataset, 1, augment=aug)
    two_img_2aug_bricon_results[name] = extractor_model.evaluate(extractor_val)

print(two_img_2aug_bricon_results)

9504/9504 [==============================] - 172s 18ms/step - loss: 5.5964 - accuracy: 0.3455
{'no augs': [0.4184468984603882, 0.8894149661064148], 'brightness/contrast - vertical flip': [0.9781972765922546, 0.7733585834503174], 'brightness/contrast - rotate': [0.9934601783752441, 0.7688341736793518], 'brightness/contrast - horizontal flip': [0.9552770256996155, 0.7798821330070496], 'brightness/contrast - gamma': [1.0565626621246338, 0.7613636255264282], 'brightness/contrast - grey': [7.770430564880371, 0.09343434125185013], 'brightness/contrast - blur': [5.5964484214782715, 0.3455387353897095]}


In [3]:
two_img_aug_results = {}
extractor_model = keras.models.load_model("models/carabid/CARABID-2-CLASSIFIER/classifier/savefile.hdf5")
for name, aug in augs.items():
    _, extractor_val = prep_data_augmented(dataset, val_dataset, val_dataset, 1, 2, augment=aug)
    two_img_aug_results[name] = extractor_model.evaluate(extractor_val)

print(two_img_aug_results)

9504/9504 [==============================] - 210s 22ms/step - loss: 0.6649 - accuracy: 0.8253
{'no augs': [0.3565865457057953, 0.9150883555412292], 'horizontal flip': [0.34561318159103394, 0.9179292917251587], 'vertical flip': [0.3435118794441223, 0.9170875549316406], 'rotate': [0.3484501540660858, 0.9158248901367188], 'brightness/contrast': [0.430027574300766, 0.8885732293128967], 'gamma': [0.3725558817386627, 0.911090075969696], 'grey': [0.6969899535179138, 0.8122895359992981], 'blur': [0.6648714542388916, 0.8253366947174072]}


In [4]:
two_img_2aug_hori_results = {}
augs = {"no augs": None,
        "horizontal flip - vertical flip": A.Compose([A.HorizontalFlip(p=1.0), A.VerticalFlip(p=1.0)]),
        "horizontal flip - rotate": A.Compose([A.HorizontalFlip(p=1.0), A.Rotate(p=1.0)]),
        "horizontal flip - brightness/contrast": A.Compose([A.HorizontalFlip(p=1.0), A.RandomBrightnessContrast(p=1.0)]),
        "horizontal flip - gamma": A.Compose([A.HorizontalFlip(p=1.0), A.RandomGamma(p=1.0)]),
        "horizontal flip - grey": A.Compose([A.HorizontalFlip(p=1.0), A.ToGray(p=1.0)]),
        "horizontal flip - blur": A.Compose([A.HorizontalFlip(p=1.0), A.Blur(p=1.0)])}
for name, aug in augs.items():
    _, extractor_val = prep_data_augmented(dataset, val_dataset, val_dataset, 1, 2, augment=aug)
    two_img_2aug_hori_results[name] = extractor_model.evaluate(extractor_val)

print(two_img_2aug_hori_results)

9504/9504 [==============================] - 195s 20ms/step - loss: 0.6578 - accuracy: 0.8275
{'no augs': [0.3565862476825714, 0.9150883555412292], 'horizontal flip - vertical flip': [0.3404251039028168, 0.9189814925193787], 'horizontal flip - rotate': [0.3519440293312073, 0.9152988195419312], 'horizontal flip - brightness/contrast': [0.4085979461669922, 0.8986742496490479], 'horizontal flip - gamma': [0.3601849377155304, 0.9144570827484131], 'horizontal flip - grey': [0.6923030018806458, 0.8142887353897095], 'horizontal flip - blur': [0.657783031463623, 0.8275462985038757]}


In [5]:
two_img_2aug_gamma_results = {}
augs = {"no augs": None,
        "gamma - vertical flip": A.Compose([A.RandomGamma(p=1.0), A.VerticalFlip(p=1.0)]),
        "gamma - rotate": A.Compose([A.RandomGamma(p=1.0), A.Rotate(p=1.0)]),
        "gamma - brightness/contrast": A.Compose([A.RandomGamma(p=1.0), A.RandomBrightnessContrast(p=1.0)]),
        "gamma - horizontal flip": A.Compose([A.RandomGamma(p=1.0), A.HorizontalFlip(p=1.0)]),
        "gamma - grey": A.Compose([A.RandomGamma(p=1.0), A.ToGray(p=1.0)]),
        "gamma - blur": A.Compose([A.RandomGamma(p=1.0), A.Blur(p=1.0)])}
for name, aug in augs.items():
    _, extractor_val = prep_data_augmented(dataset, val_dataset, val_dataset, 1, 2, augment=aug)
    two_img_2aug_gamma_results[name] = extractor_model.evaluate(extractor_val)

print(two_img_2aug_gamma_results)

9504/9504 [==============================] - 185s 19ms/step - loss: 0.6622 - accuracy: 0.8233
{'no augs': [0.3565863370895386, 0.9150883555412292], 'gamma - vertical flip': [0.3572997748851776, 0.9138257503509521], 'gamma - rotate': [0.3622608780860901, 0.9120370149612427], 'gamma - brightness/contrast': [0.42359060049057007, 0.8912037014961243], 'gamma - horizontal flip': [0.3557412326335907, 0.913615345954895], 'gamma - grey': [0.6998995542526245, 0.8103955984115601], 'gamma - blur': [0.6621925830841064, 0.8233375549316406]}


In [6]:
two_img_2aug_bricon_results = {}
augs = {"no augs": None,
        "brightness/contrast - vertical flip": A.Compose([A.RandomBrightnessContrast(p=1.0), A.VerticalFlip(p=1.0)]),
        "brightness/contrast - rotate": A.Compose([A.RandomBrightnessContrast(p=1.0), A.Rotate(p=1.0)]),
        "brightness/contrast - horizontal flip": A.Compose([A.RandomBrightnessContrast(p=1.0), A.HorizontalFlip(p=1.0)]),
        "brightness/contrast - gamma": A.Compose([A.RandomBrightnessContrast(p=1.0), A.RandomGamma(p=1.0)]),
        "brightness/contrast - grey": A.Compose([A.RandomBrightnessContrast(p=1.0), A.ToGray(p=1.0)]),
        "brightness/contrast - blur": A.Compose([A.RandomBrightnessContrast(p=1.0), A.Blur(p=1.0)])}
for name, aug in augs.items():
    _, extractor_val = prep_data_augmented(dataset, val_dataset, val_dataset, 1, 2, augment=aug)
    two_img_2aug_bricon_results[name] = extractor_model.evaluate(extractor_val)

print(two_img_2aug_bricon_results)

9504/9504 [==============================] - 211s 22ms/step - loss: 0.6691 - accuracy: 0.8270
{'no augs': [0.3565863072872162, 0.9150883555412292], 'brightness/contrast - vertical flip': [0.40406107902526855, 0.8972011804580688], 'brightness/contrast - rotate': [0.41855984926223755, 0.8954124450683594], 'brightness/contrast - horizontal flip': [0.4234601557254791, 0.8935185074806213], 'brightness/contrast - gamma': [0.4325540065765381, 0.8887836933135986], 'brightness/contrast - grey': [0.7042424082756042, 0.8097643256187439], 'brightness/contrast - blur': [0.6690558195114136, 0.8270202279090881]}
